# Import modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.fftpack as fft
import poppy
import gpipsfs
import pytz
import time
import re

from scipy import signal
from astropy.io import fits
from scipy import optimize

# Telescope dimensions

In [ ]:
outD = 7.77010            # primary diameter (m)
inD = 1.024               # inner M2 diameter (m)
n = 48                    # number sample points across the screen (Not the number of subapertures)
nacross = 43              # number of subapertures across the aperture
pscale = outD/(nacross)   # pixel size (m) of samples in pupil plane

# Make spatial frequency grid

In [ ]:
kx = fft.fftshift(fft.fftfreq(n,pscale))
ky = fft.fftshift(fft.fftfreq(n,pscale))
mg = np.meshgrid(kx,ky)
kr = np.sqrt(np.sum((m**2 for m in mg))) 

# Phase Sample

1. name_list - list containing telemetry date strings
2. fname_list - list containing path to telemetry files

In [ ]:
file1 =  '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Reduced/20160229/aored_When_2016.2.29_5.33.19_poldm_phase.fits'
file2 =  '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Reduced/20160227/aored_When_2016.2.27_0.2.8_poldm_phase.fits'

In [ ]:
def path_manager(desktop_name):
    
    if desktop_name == 'gpi_cruncher':
        rootdir = '/home/sda/mtallis/PhaseScripts/aotelem/Reduced/'
        save_path = '/home/sda/mtallis/Results/c_Eri/'
        samples_path = '/home/sda/mtallis/samples/c_Eri_samples.txt'
    
    if desktop_name == 'kipac':
        rootdir = '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Results/c_Eri/'
        save_path = '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Results/c_Eri/'
        samples_path = '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/datatables/'
        
    if desktop_name == 'laptop':
        rootdir = '/Users/melisatallis/Documents/Research/GPIDomeSeeing/data/Results/c_Eri/'
        save_path = '/Users/melisatallis/Documents/Research/GPIDomeSeeing/data/Results/c_Eri/'
        samples_path = '/Users/melisatallis/Documents/Research/GPIDomeSeeing/data/Results/c_Eri/c_Eri_samples.txt'

    dstr = time.strftime('%Y%m%d')
    return rootdir,save_path, samples_path, dstr

In [ ]:
rootdir, save_path, samples_path, dstr = path_manager('kipac')

In [ ]:
with open(samples_path,'r') as f:
    sample = f.read().splitlines() #outputs as a list of strings

Only use when fits files are available, which are typically stored in GPI cruncher

In [ ]:
fname_list = list()
name_list = list()

for i in sample:
    for root, dirs, files in os.walk(rootdir):
        for name in files:
            (base,ext) = os.path.splitext(name)
            if (ext in ('.fits')) and (i in base):
                full_name = os.path.join(root,name)
                #print(full_name)
                fname_list.append(full_name)  
                name_list.append(base[11:-12])

# Analysis Functions

1. processs_phase - Imports .fits & removes static aberrations in each phase  
2. radialProfile - Computes mean inside each radial bin starting from center
3. sp_power_spec - Computes time average of 2d DFT^2  
4. temp_power_spec - Computes actuator timeseries DFT^2 and averages over aperture
5. linear fit - fits a power law to psd. Behaves like a line in loglog space

In [ ]:
def import_fits(filepath):

    hdulist = fits.open(filepath,memmap=True)
    phase = hdulist[0].data.astype('float')

    return phase

In [106]:
def remove_zernikes(phase):
    
    m1 = gpipsfs.GeminiPrimary().sample(npix=48)
    avg_phase = np.mean(phase*m1,axis=0)
    
    z_basis = poppy.zernike.zernike_basis_faster(nterms= 6, npix = 48)
    z_coeff = poppy.zernike.opd_expand_nonorthonormal(avg_phase, aperture=m1, nterms=6)
    thin_lens = np.sum(z_coeff[:,None,None]*z_basis[:,:,:],axis=0)
    c_phase = (phase - thin_lens[None,:,:])*m1
    c_phase[np.isnan(c_phase)]=0.

    plt.imshow(c_phase[0])
    return c_phase

In [ ]:
def radialProfile(image, center=None):
    """
    Calculate the avearge radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    
    """
    ## Calculate the indices from the image
    y,x = np.indices((image.shape)) # first determine radii of all pixels
    
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
     
    r = np.hypot(x - center[0], y - center[1]).astype(np.int) 
    
    n = np.bincount(r.ravel())
    sy = np.bincount(r.ravel(), image.ravel())
    mean = sy/n
    
    return mean

In [ ]:
def convert_to_modal_basis(phase):
    
    timesteps, phx, phy = phase.shape 
    ap = gpipsfs.GPI_Apodizer().sample(npix = phx) # windowing function to smooth-out hard edges of aperture
    
    phFT = np.zeros((timesteps,phx,phy), dtype=complex)
    for t in np.arange(timesteps):
        phFT[t,:,:] = np.fft.fftshift(fft.fft2(phase[t,:,:]*ap)/np.sqrt(ap.sum()))
    print('Done with FT')

    # remove static aberrations from the signal 
    mft = np.mean(phFT,axis = 0) 
    phFT = phFT - mft[None,:,:]  
    
    return phFT

In [ ]:
def sp_power_spec(phFT):    
    
    timesteps, phx, phy = phFT.shape
    
    # compute 2d psd cube
    psd2D = np.zeros((timesteps, phx, phy),dtype=float)
    for k in np.arange(phx):
        for l in np.arange(phy):
            psd2D[:,k,l] = np.square(np.abs(phFT[:,k,l]))
    
    varpsd = np.sum(psd2D, axis=0)
    print('Done with PSD')    
    
    # compute radial average of 2d psd cube and frequency
    psd1D =  radialProfile(varpsd)
    
    return psd1D

In [ ]:
def temp_power_spec(Y):
    
    n = len(Y)
    P = np.fft.rfft(Y)
    norm = 2.0/n
    P = P * norm
    P2 = np.square(np.abs(P))
    
    return P2

In [ ]:
def linear_fit(k,Y,low_b,up_b):

    par = np.polyfit(np.log10(k[(k>low_b) & (k<up_b)]), np.log10(Y[(k>low_b) & (k<up_b)]), 1)
    slope = par[0]
    intercept = par[1]
    print(slope,intercept)
    
    return slope, intercept 

# Process Phases

Define spatial frequncy grid - has 34 bins

In [ ]:
k = radialProfile(kr)

Store spatial PSD values in DataFrame for later analysis

In [ ]:
df_sp_psd = pd.DataFrame(columns = name_list)

i=0
for file in fname_list:
    print(name_list[i])
    y_2D = import_fits(file)
    y_ft = convert_to_modal_basis(y_2D) # mFT get subtracted from fourier modes
    y_psd = sp_power_spec(y_ft)
    df_sp_psd[name_list[i]] = pd.Series(y_psd)
    i=i+1

In [ ]:
df_sp_psd.to_csv(save_path+'sp_psd'+'_'+dstr+'.txt')

Store temporal PSD values in dataframe for later analysis

In [ ]:
df_t_psd = pd.DataFrame(columns = name_list)

i=0
for file in fname_list[0:2]:
    print(name_list[i])
    y_2D = import_fits(file)
    c_y_2D = remove_zernikes(y_2D)
    y_1D = np.sum(c_y_2D,axis=(1,2))
    y_psd = temp_power_spec(y_1D)
    y_smoothed = 10**signal.savgol_filter(np.log10(y_psd), 101, 5)
    df_t_psd[name_list[i]] = pd.Series(y_smooth)
    i=i+1

In [ ]:
df_t_psd.to_csv(save_path+'t_psd'+'_'+dstr+'.txt')

Group Samples by night observed

In [ ]:
df = pd.DataFrame({'fname_list':fname_list,'name_list':name_list})
df['dts'] = pd.to_datetime(df['name_list'],format='%Y.%m.%d_%H.%M.%S',utc=True)
df['delta_t'] = df['dts']-df['dts'].shift(1)
df.loc[0,'delta_t'] = pd.Timedelta(0)

def bin_f(row):
    if row['delta_t'] > pd.Timedelta('20H'):
        return row['dts'].date()
    
df['bin']=df.apply(bin_f,axis = 1)
df.loc[0:1,'bin']=df.loc[0,'dts'].date()
df['bin']=df['bin'].fillna(method='ffill')
g_df = df.groupby('bin')

Create a dataframe with the average psd measured in each night window. Column names contain date at the start of the night 

In [ ]:
df_avg_sp_psd = pd.DataFrame()

for name,group in g_df:
    df_avg_sp_psd.loc[:,str(name)] = df_sp_psd.loc[:,group['name_list'].tolist()].mean(axis=1)

In [ ]:
df_avg_sp_psd.to_csv(save_path+'avg_sp_psd_'+dstr+'.txt')

In [ ]:
f_avg_t_psd = pd.DataFrame()

for name,group in g_df:
    df_avg_t_psd.loc[:,str(name)] = df_t_psd.loc[:,group['name_list'].tolist()].mean(axis=1)

In [ ]:
df_avg_t_psd.to_csv(save_path+'avg_t_psd_'+dstr+'.txt')

Fit power law to avg psds

In [ ]:
df_psd_results = pd.DataFrame(columns=['sp_psd_slope','sp_psd_int','sp_psd_area','t_psd_slope','t_psd_int','t_psd_area'], index = df_avg_sp_psd.columns)

for i in df_avg_sp_psd.columns:
    y = df_avg_sp_psd[i]
    x = radialProfile(kr)
    df_psd_results.loc[i,['sp_psd_slope','sp_psd_int']] = linear_fit(x,y,0.33,1.0)

In [ ]:
for i in df_avg_sp_psd.columns:
    y = df_avg_t_psd[i].dropna()
    x = fft.fftfreq(len(y),.001)
    df_psd_reseults.loc[i,['t_psd_slope','t_psd_int']] = linear_fit(x,y,2.0,30.0)

# Read all the CSV files and concatentae all of them

In [ ]:
allFiles = glob.glob(data_path + "/sp_psd_summary*.csv")

list_ = []

for file_ in allFiles[0:3]:
    df = pd.read_csv(file_,index_col=0)
    list_.append(df)

frame = pd.concat(list_, axis = 0, ignore_index = True)

In [ ]:
txt_file2 = pd.read_csv(data_path + 'sp_psd_summary20181204.csv',index_col=0) 
txt_file2_copy = copy.copy(txt_file2)

sp_psd_data = pd.DataFrame(txt_file2_copy)
sp_psd_data['dts'] = pd.to_datetime(sp_psd_data.loc[:,'filename'].str.split('_').str.get(2) + '_' + sp_psd_data.loc[:,'filename'].str.split('_').str.get(3),
              utc=True, format = '%Y.%m.%d_%H.%M.%S')

sp_psd_data['dts'] = sp_psd_data['dts'].dt.tz_localize(pytz.UTC)

In [ ]:
sp_psd_data

# Filter data

In [ ]:
def filter_data(data, imag = 10., tau = 1., see = 2., all_cond = True):
    "This selects all rows of input dataframe that satisfy conditions set by the user"
    
    date1 = datetime.date(year = 2014, month = 1, day =1)
    
    #  AO system requirements
    cond1 = data['dts'] > date1 
    cond2 = data['COADDS'] == 1
    cond3 = (data['OBSMODE'] == 'H_coron')|(data['OBSMODE'] == 'Spec')
    cond4 = data['AOFRAMES'] == 1000

    #  Good seeing conditions
    cond5 = data['IMAG'] < imag #Bright Stars
    cond6 = data['MASSTAU'] > tau  # slow moving turbulence [ms]
    cond7 = data['DIMMSEE'] < see  # smaller scale turbulence ["]
    
    if all_cond:
        ind = np.where(cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7)[0]  
        print len(ind)
    else:
        ind = np.where(cond1 & cond2 & cond3 & cond4 & cond5)[0]  
        print len(ind)

    filtered_data = data.iloc[ind]
    #filtered_data = filtered_data.dropna(subset = ['CONTR040','cal_wfe','M1_avg','TAMBIENT','Outside_OE_temperature'])
    
    new_ind = np.arange(len(filtered_data))
    filtered_data = filtered_data.set_index(new_ind)
        
    return filtered_data

In [ ]:
r = filter_data(raw_contrast_data,all_cond=True)

# Merge PSD data with IFS data

In [ ]:
sp_psd_raw_IFS_data =  pd.merge_asof(sp_psd_data[['slope','dts','filename','path']].sort_values('dts'),raw_contrast_data, on='dts',tolerance=pd.Timedelta('1min'))

In [ ]:
sp_psd_raw_IFS_data = sp_psd_raw_IFS_data.dropna(subset=['slope','M1_avg'])

In [ ]:
print sum(np.isfinite(sp_psd_raw_IFS_data['TAMBIENT']))

With grouping

In [ ]:
grouped = sp_psd_raw_IFS_data.groupby(['night_number', 'OBJNAME'], as_index= False).mean().dropna(subset = ['slope','M1_avg'])

In [ ]:
less = np.where(np.abs(grouped['M1_avg']-grouped['T_twr'])<1.)[0]
more = np.where(np.abs(grouped['M1_avg']-grouped['T_twr'])>=1.)[0]

grouped.loc[more,'slope'].mean()

In [ ]:
%matplotlib notebook

x = grouped['M1_avg']-grouped['TAMBIENT']
y = grouped['slope']
print(sum(np.isfinite(y)))


plt.figure(figsize=[8,6])
plt.scatter(x,y,c='red',marker='s')
plt.ylabel('log of Spatial PSD Slope',fontsize=15)
plt.xlabel('Primary - Outside air [C]',fontsize=15)
#plt.colorbar()


#plt.savefig(save_path+'sp_psd_slope_vs_delT_grouped_20181205.pdf')


#plt.savefig(save_path+'sp_psd_slope_vs_delT_night_20181201.pdf')
#plt.plot(sp_psd_raw_IFS_data['slope'],'.',alpha=.5)

In [ ]:
ind = np.where(grouped['slope'] == np.max(grouped['slope']))[0]

In [ ]:
def search_for_dome_seeing(data, sample_size, presence = True):
    
    cut = 0
    sample = set()
    
    if presence:
        
        while len(sample) <= sample_size:
            tau0 = data.MASSTAU.sort_values(ascending = False)[0:cut]
            mirror_to_air_temp = np.abs(data.M1_avg - data.T_twr).sort_values(ascending = False)[0:cut]
            contrast = data.CONTR040.sort_values(ascending = False)[0:cut]
            psd_slope = data.slope.sort_values(ascending = False)[0:cut]
            sample = set(tau0.index) & set(mirror_to_air_temp.index) & set(contrast.index) & set(psd_slope.index)
            cut = cut + 1
            
    else:
        
        while len(sample) <= sample_size:
            tau0 = data.MASSTAU.sort_values(ascending = False)[0:cut]
            mirror_to_air_temp = np.abs(data.M1_avg - data.T_twr).sort_values()[0:cut]
            contrast = data.CONTR040.sort_values()[0:cut]
            psd_slope = data.slope.sort_values(ascending = True)[0:cut]
            sample = set(tau0.index) & set(mirror_to_air_temp.index) & set(contrast.index) & set(psd_slope.index)
            cut = cut + 1
    return list(sample)  

In [ ]:
sp_psd_raw_IFS_data.loc[search_for_dome_seeing(sp_psd_raw_IFS_data,10,presence=True),
                       ['dts','CONTR040','M1_avg','TAMBIENT','MASSTAU','slope','IMAG','filename']].sort_values("CONTR040",ascending = True)

In [ ]:
sp_psd_raw_IFS_data.loc[search_for_dome_seeing(sp_psd_raw_IFS_data,10,presence=False),
                       ['dts','CONTR040','M1_avg','TAMBIENT','MASSTAU','slope','filename']].sort_values("CONTR040",ascending = False)